In [1]:
## Import statements

%matplotlib inline

import xarray as xr
import intake
import cf_units as cf
import cartopy
import pandas as pd
import cartopy.crs as ccrs
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as colors
from cartopy.util import add_cyclic_point
from IPython.display import display, clear_output

# util.py is in the local directory
# it contains code that is common across project notebooks
# or routines that are too extensive and might otherwise clutter
# the notebook design
import util 

if util.is_ncar_host():
    col = intake.open_esm_datastore("../catalogs/glade-cmip6.json")
else:
    col = intake.open_esm_datastore("../catalogs/pangeo-cmip6.json")

/srv/conda/envs/notebook/lib/python3.7/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['stac-catalog', 'stac-collection', 'stac-item'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


In [ ]:
def plot_trend_map(X):
# this function plots the trend in a
# 3d xarray dataset, where the dimenison are lat, lon, and time.
# the trend is the defined of the slope of the best-fit line for 
# the equation:
# y = at+b
# from simple linear regresiion, where y is the fitted value of
# the input "var", a is the slope, and b is the intercept

# input: X, a 3d xarray dataset with dimensions lat, lon, and time
# output: 

# extract lat, lon, and time from dataset
    lat = X.lat
    lon = X.lon

##
# Time is extracted as such only because I needed floating point values as the regressors
# I suspect there is a much better way to convert date-time format to, e.g., days since the first model date
# Please let me know of any suggestions
##
    timest=X.isel(time=0)
    timearr = xr.Dataset({'realtime': ('time', np.arange(len(arr.time))), 'time': arr.time})
    times=timearr.realtime


    n=len(times)
    df = n-2 # degrees of freedom


    Xsq = X**2
    Xmean = X.mean(dim='time')
    timemean = times.mean(dim='time')
    timeanom = times-timemean #  anomoly in time array
    Xanom = X - Xmean # anomoly in X
    Xanomsq = Xanom**2 # variance in X
    Xxtimeanom = Xanom*timeanom 
    sumanom = Xxtimeanom.sum(dim='time') # covariance in X, time
    timeanomsq = timeanom**2
    sumtimeanomsq = timeanomsq.sum(dim='time') # variance in time

# slope and intercept
    slope = sumanom/sumtimeanomsq
    intercept = Xmean - slope*timemean
    slopes = slope.compute()
    intercepts = intercept.compute()

# best fit value
    Xfit = slope*X + intercept
    Xfit=Xfit.transpose('time','lat','lon')
    resid = X-Xfit # residuals
    residsq = resid**2

    varbest = residsq/(df) # best guess estimator of variance in X

#  standard errors of slopes and intercepts
    slope_var = varbest/sumtimeanomsq
    intercept_var = ((1/n) + ((timemean**2)/sumtimeanomsq))*varbest
    
    slope_std = slope_var**0.5
    intercept_std = intercept_var**0.5
    
    slope_stds = slope_std.compute()
    intercept_stds = intercept_std.compute()
    
# calculate r^2 and p values
    Total_SS = Xanomsq.sum(dim='time')
    Regression_SS = (sumanom**2)/sumtimeanomsq
    Rsq = Regression_SS/Total_SS
    r=Rsq**0.5
    r=r.compute()
    t = r * (df / ((1 - r) * (1 + r)))**0.5
    p = stats.distributions.t.sf(abs(t), df)

# convert p values to an xarray dataset
    parr = xr.ones_like(Xmean)
    parr.name = 'p value'
    parr.data = p

################################
# plot slopes
#################################
    fig = plt.figure(figsize=(14, 20))


# normalization of colobar
    norm = colors.DivergingNorm(vmin=slopes.min(), vmax=slopes.max(), vcenter=0.)

# add cyclic point

        
    slopes, lon = add_cyclic_point(slopes[:, :], coord=slopes.lon)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())

# filled contours
    cf = ax.contourf(lon, lat, slopes, norm=norm, cmap='bwr', transform=ccrs.PlateCarree());

# land
    land = ax.add_feature(cartopy.feature.NaturalEarthFeature('physical','land','110m', facecolor='black'))

# colorbar and labels
    cb = plt.colorbar(cf, shrink=0.3)
    cb.ax.set_title(X.units+'/unit time')
    ax.set_title('slopes'+' '+X.name);


# output dictionary for slopes, intercepts, standard errors for slope and intercept, r, and p-value
    D = dict()
    D['slopes'] = slopes;D['intercepts']=intercepts;
    D['slope error']=slope_stds;D['intercept error']=intercept_stds
    D['r'] = r;D['p value']=parr
